### What is the longest match recorded in terms of duration?

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

##### MySQL Solution

~~~sql
SELECT t.match_id,
(t.period_1+t.period_2+t.period_3+t.period_4+t.period_5) AS match_duration,
wp.slug AS winner_player, lp.slug AS loser_player,
tr.tournament_name, FROM_UNIXTIME(e.start_datetime, "%Y-%m-%d %H:%i") AS start_datetime
FROM MatchTimeInfo AS t
JOIN GameInfoResult AS r ON r.match_id=t.match_id
JOIN MatchTournamentInfo AS tr ON tr.match_id=t.match_id
JOIN MatchEventInfo AS e ON e.match_id=t.match_id
JOIN Players AS wp ON wp.id=r.winner_player_id
JOIN Players AS lp ON lp.id=r.loser_player_id
ORDER BY match_duration DESC LIMIT 1
~~~

In [63]:
from tennis_data_analysis.db.mysql import Database
import tabulate as tabulate

mysql = Database(os.environ.get('DB_HOST'), os.environ.get('DB_USERNAME'), os.environ.get('DB_PASSWORD'), os.environ.get('DB_DATABASE'), os.environ.get('DB_PORT'), True)

sql = """
SELECT t.match_id,
(t.period_1+t.period_2+t.period_3+t.period_4+t.period_5) AS match_duration_second,
wp.slug AS winner_player, lp.slug AS loser_player,
tr.tournament_name, FROM_UNIXTIME(e.start_datetime, "%Y-%m-%d %H:%i") AS start_datetime
FROM MatchTimeInfo AS t
JOIN GameInfoResult AS r ON r.match_id=t.match_id
JOIN MatchTournamentInfo AS tr ON tr.match_id=t.match_id
JOIN MatchEventInfo AS e ON e.match_id=t.match_id
JOIN Players AS wp ON wp.id=r.winner_player_id
JOIN Players AS lp ON lp.id=r.loser_player_id
ORDER BY match_duration_second DESC LIMIT 1
"""

mysql.db.execute(sql)
result = mysql.db.fetchone()

if result:
    result['match_duration_hour'] = "{:.2f}".format(result['match_duration_second'] / 60 / 60)
    data = [[k, result[k]] for k in result]
    print(tabulate.tabulate(data, tablefmt='outline', stralign="left", numalign="center"))
else:
    print('Not Found!')


+-----------------------+-------------------------------------+
| match_id              | 12346747                            |
| match_duration_second | 171153                              |
| winner_player         | pieri-samuele                       |
| loser_player          | vulpitta-gabriele                   |
| tournament_name       | Cervia, Singles Main, M-ITF-ITA-14A |
| start_datetime        | 2024-05-20 14:00                    |
| match_duration_hour   | 47.54                               |
+-----------------------+-------------------------------------+
